In [5]:
import numpy as np
import pandas as pd
from pyDOE2 import *

# Recursos
- [Catapulta](https://sigmazone.com/catapult/)
- [Documentação pyDOE2](https://pythonhosted.org/pyDOE/)
- [Curso de Design of Experiments da PennState](https://online.stat.psu.edu/stat503/lesson/1)
- [Tutorial de DOE com catapulta do Sigmazone](https://sigmazone.com/quantum_xl_doe_tutorial/#Step_1:_Setting_up_the_Designed_Experiment)

## Obtendo as amostras
Devido ao número de fatores e níveis, optamos por usar o Latin Hypercube Sampling (LHS) para obter amostras do nosso espaço. 

O Latin Hypercube Sampling (LHS) é uma abordagem que divide a faixa de cada fator de entrada em intervalos iguais e garante que cada intervalo seja representado exatamente por uma amostra. O LHS é frequentemente utilizado para explorar o espaço de design de forma mais eficiente do que a amostragem aleatória simples.

O código abaixo só precisou ser rodado uma vez para a obtenção do dataframe com as configurações do experimento.

In [6]:
# Define fatores e número de amostras
factor_names = ['release_angle', 'firing_angle', 'cup_elevation', 'bungee_position', 'pin_elevation']
num_factors = len(factor_names)
num_samples = 70

# Define valores máximos e mínimos para cada fator.
min_values = [90, 90, 200, 100, 100]
max_values = [185, 140, 300, 200, 200]

# # Cria as amostras
# samples = lhs(num_factors, samples=num_samples)

# # Coloca as amostras nos intervalos das configurações
# scaled_samples = min_values + samples * (np.array(max_values) - np.array(min_values))

# # Converte os valores para int
# scaled_samples = scaled_samples.astype(int)

# # Transforma as amostras em um dataframe
# df = pd.DataFrame(scaled_samples)
# df.columns = factor_names

In [7]:
# Descarta linha se release_angle < firing_angle
# for index, row in df.iterrows():
#     if (row['release_angle'] < row['firing_angle']):
#         df = df.drop(index)
# df = df.reset_index(drop=True)
# df["distance"] = ""
# df.to_csv("df_catapulta_empty.csv")

## Realizando o experimento.
Colocamos as configurações de cada linha do dataframe na [catapulta](https://sigmazone.com/catapult/) para preencher a coluna `distance` e salvá-lo como um novo dataframe.

A planilha compartilhada está disponível no [Google Sheets](https://docs.google.com/spreadsheets/d/1e-EHZcRrpu5jy7baX_Z1_XDp0cppIyoXXveRIzQKk2E/edit#gid=1081657349)

In [40]:
df = pd.read_csv("df_catapulta_complete.csv")
# df = df.dropna(ignore_index=True)
df = df.dropna(ignore_index=True).drop('Unnamed: 0', axis=1)
df['distance '] = df['distance'].astype(int)
df

,release_angle,firing_angle,cup_elevation,bungee_position,pin_elevation,distance,distance
0,183,100,211,168,120,185.0,185
1,173,124,240,110,100,125.0,125
2,128,104,243,117,163,96.0,96


## Análise dos dados
Usamos a biblioteca `pyDOE2` para realizar a análise de variância, entre outras.

## Otimização
Técnica de Análise de Superfície de Resposta (RSM) para achar a melhor configuração da catapulta.

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [42]:
# Variáveis independentes
x = df[['release_angle', 'firing_angle','cup_elevation', 'bungee_position', 'pin_elevation']]

# Variável de resposta
y = df['distance']

Geramos uma matriz polinomial de segundo grau, o que adiciona mais complexidade ao modelo considerando relações não-lineares, além de capturar relações mais complexas como curvatura e interação entre variáveis

In [45]:
poly = PolynomialFeatures(degree=2)
x_poly = poly.fit_transform(x)

In [46]:
model = LinearRegression()
model.fit(x_poly, y)

LinearRegression()